In [1]:
# Povm

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path
import itertools

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_base import ProjectedGradientDescentBase, ProjectedGradientDescentBaseOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_measurement, get_z_measurement
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)
from quara.objects.operators import tensor_product

from quara.data_analysis.simulation import SimulationSetting

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
start_all = time.time()

In [4]:
# setup system
e_sys_1 = ElementalSystem(0, get_normalized_pauli_basis())
c_sys_1 = CompositeSystem([e_sys_1])

e_sys_2 = ElementalSystem(1, get_normalized_pauli_basis())
c_sys_2 = CompositeSystem([e_sys_2])

# |+><+|
# state_x0 = get_x0_1q(c_sys)
# |+i><+i|
# state_y0 = get_y0_1q(c_sys)
# |0><0|
# state_z0 = get_z0_1q(c_sys)
# |1><1|
# state_z1 = get_z1_1q(c_sys)

tester_objects = []

# |+><+| |+i><+i| |0><0| |1><1|
func_list = [get_x0_1q, get_y0_1q, get_z0_1q, get_z1_1q]

for i, funcs in enumerate(itertools.product(func_list, func_list)):
    state1 = funcs[0](c_sys_1)
    state2 = funcs[1](c_sys_2)

    state_2qubit = tensor_product(state1, state2)

    tester_objects.append(state_2qubit)

In [5]:
true_object = tensor_product( get_z_measurement(c_sys_1),  get_z_measurement(c_sys_2))

In [7]:
num_data = [100, 1000, 10000]
n_rep = 10
measurement_n = len(true_object.vecs)  # 測定値の数

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

seed = 777
qtomography_list = [
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed),
]

para_list = [
    True,
    False,
    True,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator()
]

estimation_results_list = []
elapsed_times = []
simulation_settings = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]

    start = time.time()

    estimation_results = data_analysis.estimate(
        qtomography=qtomography,
        true_object=true_object,
        num_data=num_data,
        estimator=estimator,
        iteration=n_rep,
    )

    # stock settings of this simulation
    simulation_setting = SimulationSetting(
        name=name,
        estimator=estimator
    )
    print(simulation_setting)
    simulation_settings.append(simulation_setting)

    estimation_results_list.append(estimation_results)

    elapsed_time = time.time() - start
    print("elapsed_time: {0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

  0%|          | 0/10 [00:00<?, ?it/s]

Name: LinearEstimator(True)
Estimator: LinearEstimator
Loss: None
Algo: None
elapsed_time: 0.10451223452885945[min]



  0%|          | 0/10 [00:00<?, ?it/s]

Name: LinearEstimator(False)
Estimator: LinearEstimator
Loss: None
Algo: None
elapsed_time: 0.12541684707005818[min]



  0%|          | 0/10 [00:00<?, ?it/s]

Name: ProjectedLinearEstimator(True)
Estimator: ProjectedLinearEstimator
Loss: None
Algo: None
elapsed_time: 2.2129801472028094[min]



100%|██████████| 10/10 [01:20<00:00,  8.05s/it]

Name: ProjectedLinearEstimator(False)
Estimator: ProjectedLinearEstimator
Loss: None
Algo: None
elapsed_time: 1.341193163394928[min]



In [11]:
report.export_report("povmt_2qubit.pdf",
    estimation_results_list,
    simulation_settings,
    true_object,
    tester_objects,
    seed=seed,
    computation_time=sum(elapsed_times)
)

100%|██████████| 10/10 [00:00<00:00, 22882.18it/s]

​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...
​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
​Generating a graph for MSE ...
  LinearEstimator(True)=[9.564439999999992, 9.098220599999992, 9.002429433999996]
  LinearEstimator(False)=[0.2683000000000003, 0.026104399999999993, 0.003193570000000022]
  ProjectedLinearEstimator(True)=[4.78779152917869, 4.766868227807777, 4.741182270052572]
  ProjectedLinearEstimator(False)=[0.1667886712370791, 0.021935338039248714, 0.0020380067683603486]
  LinearEstimator(True)=[9.564439999999992, 9.098220599999992, 9.002429433999996]
  ProjectedLinearEstimator(True)=[4.78779152917869, 4.766868227807777, 4.741182270052572]
  LinearEstimator(False)=[0.2683000000000003, 0.026104399999999993, 0.003193570000000022]
  ProjectedLinearEstimator(False)=[0.1667886712370791, 0.021935338039248714, 0.0020380067683603486]


100%|██████████| 10/10 [00:00<00:00, 5414.10it/s]

​​Generating physicality violation test blocks ...


...

col_n=4
DEBUG:


4it [00:00, 96.15it/s]

4it [00:00, 83.36it/s]
 75%|███████▌  | 3/4 [00:00<00:00, 20.18it/s]
4it [00:00, 82.86it/s]
  0%|          | 0/4 [00:00<?, ?it/s]
4it [00:00, 87.12it/s]

4it [00:00, 66.94it/s]
 50%|█████     | 2/4 [00:00<00:00, 17.01it/s]
4it [00:00, 84.11it/s]

4it [00:00, 80.56it/s]
  0%|          | 0/4 [00:00<?, ?it/s]
4it [00:00, 90.45it/s]

4it [00:00, 98.05it/s]

4it [00:00, 79.45it/s]
 75%|███████▌  | 3/4 [00:00<00:00, 19.69it/s]
4it [00:00, 97.21it/s]
  0%|          | 0/4 [00:00<?, ?it/s]
4it [00:00, 86.23it/s]

0it [00:00, ?it/s]

col_n=4
DEBUG:


4it [00:00, 78.22it/s]
 50%|█████     | 2/4 [00:00<00:00, 18.68it/s]
4it [00:00, 84.85it/s]

4it [00:00, 85.35it/s]
  0%|          | 0/4 [00:00<?, ?it/s]
4it [00:00, 78.71it/s]

4it [00:00, 89.90it/s]
 50%|█████     | 2/4 [00:00<00:00, 18.72it/s]
4it [00:00, 81.83it/s]

4it [00:00, 72.22it/s]
  0%|          | 0/4 [00:00<?, ?it/s]
4it [00:00, 87.74it/s]

4it [00:00, 85.32it/s]
 50%|█████     | 2/4 [00:00<00:00, 19.24it/s]
4it [00:00, 80.77it/s]

4it [00:00, 83.35it/s]
100%|██████████| 10/10 [00:00<00:00, 145.87it/s]


Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (povmt_2qubit.pdf)


In [9]:
true_object.calc_eigenvalues()

[array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]),
 array([0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]),
 array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])]